In [1]:
import cv2
import os
import sys
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


Here initially the required libraries are initially imported to the program like **OpenCV (cv2), NumPy , Math , Sys , os .**
Then , the **Tensorflow** Module is imported followed by **Keras** from where *Sub-Modules :  Sequential , Dense, Conv2D, MaxPooling2D, Dropout, Flatten* are imported.


  

In [3]:

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        #print(logs.get('acc'))
        if(logs.get('acc')!= None and logs.get('acc')>0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

def image_resize(image, height = 45, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if(w>h):
    	r = height / float(h)
    	dim = (int(w * r), height)
    else:
    	r = height / float(w)
    	dim = (height, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

def data_read(data_path,category='skin',source='Dataset_skin'):
	labels = []
	images = []
	data_path = os.path.join(data_path,source)
	for label in os.listdir(data_path):
		label_path = os.path.join(data_path,label,category)
		for image in os.listdir(label_path):
			img = cv2.imread(os.path.join(label_path,image))
			if(img is not None):
				resized_img = image_resize(img)
				images.append(resized_img)
				labels.append(label)
	return labels, images


From the **Keras** Module the method **on_epoch_end()** from the class **myCallback(tf.keras.callbacks.Callback)** is overloaded and updated as per the requirements , such that the training of the model automatically stops on reaching a accuracy of 99%

The function **image_resize()** is defined which resizes the input image to the model for training as *45*45* image , and returns the resized image as a output . 

The function **data_read()** reads all the images from the categorical folfer named 'skin' under the directory  names "Dataset" and stores the images in the form of array . the collection of the array_of_images is stored under the variable named *images* and labels store the alphabet they are representing .


*Example ==>* An image represting 'A ' is stored as :

   *Image Data* :    [ [ [ 15 , 15 , 16 ] , [ 22 , 22 , 24 ] , [ 1 , 0 , 1 ] , [ 0 , 0 . . . . ] 
   
   *Label*      :     A

and the same is returned as a Numpy array of Image arrays called *images* and a list of respective denoying alphabet as *labels* 


In [ ]:
labels, images = data_read(r'C:\Users\ndh60048\Documents\ASL translator')
data = pd.DataFrame({'Image Data':images,'Label':labels})
print('Data Sample')
print(data.head(10))


label_cats = np.unique(labels)
int_encoding = np.arange(len(label_cats))
label_to_int = pd.DataFrame({'Label':label_cats,'Encoded':int_encoding})
label_to_int.to_csv('label_encoded.csv')

onehot_encoder = OneHotEncoder(sparse=False)
labels_arr = np.array([[i] for i in labels]).reshape(len(labels),1)
labels_encoded = onehot_encoder.fit_transform(labels_arr)


initially  , The data_read() is called by passing the path of the Dataset Folder as a argument to the function.
and labels are encoded as integers and stored in a .csv file

the encoded labels are then passed through the *OneHotEncoder()* , which one_hot_encodes the labels.


Example :
 Label - **D**
 
 after OneHotEncoding becomes : 
 **array([0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ] )**



In [2]:
#epoch = 135

model = Sequential()
print(np.shape(images[0]))
model.add(Conv2D(32,(3,3),padding='same',activation='relu',input_shape=np.shape(images[0])))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(len(labels_encoded[0]),activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 200
callbacks = myCallback()   #object of the class created
history = model.fit(np.array(images), labels_encoded, epochs=epochs, batch_size= 100 , callbacks=[callbacks]) # 32)

model.save('CNN_MODEL.h5') 

(45, 45, 3)
Epoch 1/200
11645/11645 [==============================] - 168s 14ms/step - loss: 3.1507 - accuracy: 0.1717
Epoch 2/200
11645/11645 [==============================] - 160s 14ms/step - loss: 0.9788 - accuracy: 0.6927
Epoch 3/200
11645/11645 [==============================] - 157s 13ms/step - loss: 0.3866 - accuracy: 0.8769
Epoch 4/200
11645/11645 [==============================] - 154s 13ms/step - loss: 0.2226 - accuracy: 0.9299
Epoch 5/200
11645/11645 [==============================] - 148s 13ms/step - loss: 0.1569 - accuracy: 0.9509
Epoch 6/200
11645/11645 [==============================] - 141s 12ms/step - loss: 0.1233 - accuracy: 0.9597
Epoch 7/200
11645/11645 [==============================] - 141s 12ms/step - loss: 0.0996 - accuracy: 0.9703
Epoch 8/200
11645/11645 [==============================] - 143s 12ms/step - loss: 0.0771 - accuracy: 0.9755
Epoch 9/200
11645/11645 [==============================] - 143s 12ms/step - loss: 0.0698 - accuracy: 0.9785
Epoch 10/200
116

11645/11645 [==============================] - 130s 11ms/step - loss: 0.0018 - accuracy: 0.9994
Epoch 151/200
11645/11645 [==============================] - 133s 11ms/step - loss: 0.0076 - accuracy: 0.9978
Epoch 152/200
11645/11645 [==============================] - 131s 11ms/step - loss: 0.0090 - accuracy: 0.9978
Epoch 153/200
11645/11645 [==============================] - 131s 11ms/step - loss: 0.0069 - accuracy: 0.9983
Epoch 154/200
11645/11645 [==============================] - 132s 11ms/step - loss: 0.0086 - accuracy: 0.9979
Epoch 155/200
11645/11645 [==============================] - 129s 11ms/step - loss: 0.0072 - accuracy: 0.9982
Epoch 156/200
11645/11645 [==============================] - 129s 11ms/step - loss: 0.0064 - accuracy: 0.9984
Epoch 157/200
11645/11645 [==============================] - 134s 12ms/step - loss: 0.0116 - accuracy: 0.9972
Epoch 158/200
11645/11645 [==============================] - 138s 12ms/step - loss: 0.0034 - accuracy: 0.9987
Epoch 159/200
11645/1164

from *keras.models* , an instance of the Sequential model is created .
then using model.add() , convolution filters are added to the model as *Conv2D()* with activation function as *relu*.
Therby ,  3 hidden convolution layers and 28 output categorical classes are added to model .

*model.summary()* gives the backbone and architecture of the model to be compiled

The model is compiled as *model.compile()*  with optimizer as 'adam'and loss as 'categorical_crossentropy'
Finally , the model is fitted using *model.fit()* , as per the specified arguments here the epoch is set to as 200 by which the training of the model takes place for 200 epochs and is training is cancelled as per the callback defined before .
The model is finally saved as *CNN_MODEL.h5* using *model.save()*

In [12]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 45, 45, 32)        896       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 43, 43, 32)        9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 21, 21, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 19, 19, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 9, 9, 64)         